In [4]:
import wave
import numpy as np
import os

def pcm_to_wav_chunks(pcm_file_path, wav_file_base_path, sample_rate, num_channels, max_wav_size=2**31-1):
    bit_depths = [ 
                  16,
                #   64
                  ]
    # Перебираемые битовые глубины
    chunk_size = 1024 * 1024 * 1024  # Размер чанка для чтения данных (1 GB)

    for bits_per_sample in bit_depths:
        try:
            # Определение типа данных на основе битовой глубины
            if bits_per_sample == 8:
                dtype = np.uint8  # 8-битные данные без знака
            elif bits_per_sample == 16:
                dtype = np.int16  # 16-битные данные со знаком
            # elif bits_per_sample == 32:
            #     dtype = np.int32  # 32-битные данные со знаком
            elif bits_per_sample == 64:
                dtype = np.float64  # 64-битные данные с плавающей точкой
            else:
                raise ValueError("Неподдерживаемая битовая глубина")

            # Создание папки для текущего файла
            output_dir = f"{wav_file_base_path}_bitdepth{bits_per_sample}"
            os.makedirs(output_dir, exist_ok=True)

            # Чтение данных из .pcm файла и запись в .wav файлы порциями
            with open(pcm_file_path, 'rb') as pcm_file:
                chunk_index = 0
                while True:
                    raw_data = pcm_file.read(chunk_size)
                    if not raw_data:
                        break

                    audio_data = np.frombuffer(raw_data, dtype=dtype)

                    # Если данные с плавающей точкой, нормализуем и преобразуем в int16
                    if bits_per_sample == 64:
                        audio_data = (audio_data * 32767).astype(np.int16)

                    wav_file_path = os.path.join(output_dir, f"part{chunk_index}.wav")
                    with wave.open(wav_file_path, 'wb') as wav_file:
                        wav_file.setnchannels(num_channels)
                        wav_file.setsampwidth(2 if bits_per_sample == 64 else bits_per_sample // 8)
                        wav_file.setframerate(sample_rate)
                        wav_file.writeframes(audio_data.tobytes())

                    # Проверка размера файла
                    if os.path.getsize(wav_file_path) > max_wav_size:
                        print(f"Файл {wav_file_path} превышает максимальный размер и будет разбит на части.")
                        os.remove(wav_file_path)
                        break

                    print(f"Файл {wav_file_path} успешно создан.")
                    chunk_index += 1

        except Exception as e:
            print(f"Ошибка при обработке битовой глубины {bits_per_sample}: {e}")

# Пример использования
dir_file = "/home/pyatanin/VSproj/30.09.2024/august_records/"
dir_out = "/home/pyatanin/VSproj/30.09.2024/august/"
filename_List = [
    'output_test_2_10min.pcm',
    # 'test_2_10min.pcm',
    'output_record_3.pcm',
]
sample_rate = 16000  # Частота дискретизации в Гц
num_channels = 4     # Количество каналов

for file_name in filename_List:
    pcm_file_path = dir_file + file_name
    wav_file_base_path = os.path.join(dir_out, file_name.replace('.pcm', ''))
    pcm_to_wav_chunks(pcm_file_path, wav_file_base_path, sample_rate, num_channels)

Файл /home/pyatanin/VSproj/30.09.2024/august/output_test_2_10min_bitdepth16/part0.wav успешно создан.
Файл /home/pyatanin/VSproj/30.09.2024/august/output_record_3_bitdepth16/part0.wav успешно создан.
Файл /home/pyatanin/VSproj/30.09.2024/august/output_record_3_bitdepth16/part1.wav успешно создан.
